# TimeSeries

Innanzitutto devo mettere i dati da qualche parte

In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
from pyburst import EventDataLoader

data = EventDataLoader.get_event_data(
    ["GW150914", "GW190814"],
    ["L1"],
    verbose=True,
    duration=20.,
)

gwpy_timeseries = data["GW150914"]["L1"]["time_series"]

strain = gwpy_timeseries.value
times = gwpy_timeseries.times.value

Fetched 1 URLs from gwosc.org for [1126259452 .. 1126259473))
Fetched 1 URLs from gwosc.org for [1249852247 .. 1249852267))
Reading data... Reading data... [Done]
[Done]


Voglio usare un formato dati più standard, tipo xarray

In [21]:
from pyburst.timeseries import TimeSeries
import numpy

arr = TimeSeries.build(strain, "GW150914", "L1", times = times)

AssertionError: time data has a sampling rate of 4096 s, input sampling rate is 2048 s. Please correct!

In [ ]:
import xarray
from numpy import float32
import gwpy
from pyburst import type_check
import astropy
import pandas

@type_check()
def gwpy_to_xarray(
    timeserie: gwpy.timeseries.TimeSeries,
    event_gps: float,
    detector_id: str,
    name: str = "", 
    detector_name: str = "",
):
    strain = timeserie.value
    time = timeserie.times.value

    time_axis = pandas.to_datetime(astropy.time.Time(time, format='gps').iso)

    return xarray.DataArray(strain, coords=dict(time = time_axis), attrs=dict(event_name = name, detector_id = detector_id, detector_name = detector_name))

arr_timeseries = gwpy_to_xarray(gwpy_timeseries, data["GW150914"]["L1"]["gps_time"], "L1")

@xarray.register_dataarray_accessor("timeseries")
class TimeSeries:
    def __init__(self, xarray_obj):
        self._obj = xarray_obj
        self._check_array_attrs("time")

    def _check_array_attrs(self, attribute: str):
        if not hasattr(self._obj, attribute):
            raise AttributeError(
                f"Array has no attribute {attribute}, to use timeseries properties provide an array with the corret format."
            )

    @property
    def dt(self):
        return float32(self._obj.time[1] - self._obj.time[0])

C:\Users\felic\AppData\Local\Temp\ipykernel_22128\2435398839.py:25: AccessorRegistrationWarning: registration of accessor <class '__main__.TimeSeries'> under name 'timeseries' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  @xarray.register_dataarray_accessor("timeseries")
